In [1]:
# day 32
# RNN 
from keras.models import Sequential # 케라스 층생성 모듈

Using TensorFlow backend.


In [5]:
model=Sequential()
model.add(Embedding(voc, hidden_size, input_length)) # 기본적 케라스 작업방법
# 일반적 계층 추가

# model.add(Dense(2, input_dim=4, init='uniform', activation='softmax')) 
# 자연어처리 voca 생성 당시, 1000개..input_dim = 1000),init='normal') 가중치 초기화법
# 다중분류 - 출력층 softmax
# 이진분류 - sigmoid
# linear (default value)

model.add(Dense(4, input_dim=10, init='normal', activation = 'softmax'))
# 출력이 4개 입력데이터 차원 10 
model.add(Dense(1, activation='sigmoid'))
model.summary() # : 요약정보
model.compile() # : 함수설정
model.fit() # : 학습
model.evaluate() # : 모델 평가
model.predict() # : 예측
model.save()
model.load()



NameError: name 'Embedding' is not defined

In [8]:
import tensorflow as tf
tf.__version__ # tf의 버젼이 나온다
tf.get_default_graph() # 기본그래프정보 출력

In [14]:
# 내가 직접 그래프 만들어 거기에다가 노드를 지정해주자 ! 
testGraph = tf.Graph() # 그래프 클래스 객체 생성 
with testGraph.as_default(): # 사용자 그래프에 노드정의 
    x = tf.constant(10)
    y = x + 20
# testGraph


In [11]:
# 그래프 실행을 위해서 세션을 생성해줘야한다 
# Session() : 세션객체 생성, 분산처리 환경 => 통신을 연결하겠다
# 노드를 실행할수 있는 연결객체를 생성하겠다 
# run() : (그래프 구성 노드) 기입시 연산을 처리한다 => 결과를 반환
# ex) sess.run() 
# close() : session 종료문

x = tf.constant(5)
y = x + 10
sess=tf.Session()
print(sess.run(x))
print(sess.run(y))
sess.close()


5
15


In [12]:
# with 구문은 close() 문을 달아주지않아도 자동적으로 처리해준다
with tf.Session() as sess:
    print(sess.run(x))
    # 위세션이 사용하는 그래프는 default 기본그래프를 사용한다
    # 특별히 그래프를 지정하지 않았으므로, 기본그래프로 세션실행하겠다 구문 
    

5


In [15]:
with tf.Session(graph=testGraph) as sess:
    print(sess.run(x))


10


In [18]:
# tensorflow.session() 
# sess.run # 노드 실행 
# ... # 매번 하기 불편하므로 

# sess=tf.InteractiveSession()
# x.eval()
# y.eval()

In [21]:
# scope() 
# name space (이름공간)
# 폴더 (file system) 개념과 유사하다 
# 같은 폴더 내에서 같은이름은 안돼 

with tf.name_scope('ns1'):
    v3 = tf.Variable(1, name='var3')
    v4 = tf.add(v3, 2, name='var4')
print(v3.op.name)
    
    

ns1/var3


In [23]:
var1 = tf.Variable(5,name='var1') 
var2 = tf.add(var1, 5, name='var2') 
var1.op.name

'var1_3'

In [74]:
## PPT 
# 입력 4 # 출력 2 (hidden s[0])
# 'hihello'
# 이정도면 time-Step : 6, input_dim : 5, hideensize : 5
idx2char = [ 'h','i','e','l','o']
xdata = [[0,1,0,2,3,3]]
x_onehot =[[
    [1,0,0,0,0],
    [0,1,0,0,0],
    [1,0,0,0,0],
    [0,0,1,0,0],
    [0,0,0,1,0],
    [0,0,0,1,0]]]
ydata = [[1,0,2,3,3,4]] 
tf.reset_default_graph()
num_classes = 5
input_dim = 5 # 입력데이터의 차원(단어벡터의길이)
hidden_size = 5
batch_size = 1
sequence_length = 6 # hihell 을 학습시키는것
learning_rate = 0.05

x = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
y = tf.placeholder(tf.int32, [None, sequence_length])

cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size) 
# 출력되어지는 hidden size : 5 // 출력값의 갯수 ()
ini_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x, initial_state = ini_state, 
                                     dtype = tf.float32)
x_fc = tf.reshape(outputs, [-1, hidden_size]) # -1 : None(전체)
outputs = tf.contrib.layers.fully_connected(inputs=x_fc, 
                                  num_outputs=num_classes)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
                            # [1,6,5]=>[[[0.1,0.2,0.5,0.1,0.1]],[],,,,]
weights = tf.ones([batch_size, sequence_length]) # weight 중요도 조절 # 동일한 중요도
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = y, 
                                                 weights = weights)
loss = tf.reduce_mean(sequence_loss)

prediction = tf.argmax(outputs, axis=2)

train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
with tf.Session() as sess : 
    sess.run(tf.global_variables_initializer())
    for i in range(101):
        lv, _= sess.run([loss, train], feed_dict ={x:x_onehot, y:ydata})
        res=sess.run(prediction,feed_dict ={x:x_onehot})
        print('step:',i,'cost:',lv,'prediction:', res, 'true Y:',ydata)
        resStr = [idx2char[c] for c in np.squeeze(res)]
        print('prediciton str:',''.join(resStr))
        
# num_units 는 출력개수 
# logits = score

# hihell
# ex) hihell - > ihello 
# hihell - > uihello 


step: 0 cost: 1.6189046 prediction: [[0 2 2 4 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: heeooo
step: 1 cost: 1.6061963 prediction: [[2 2 2 4 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: eeeooo
step: 2 cost: 1.5868315 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: eeeeoo
step: 3 cost: 1.5669628 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: eeeeoo
step: 4 cost: 1.5459489 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: eeeeoo
step: 5 cost: 1.5248747 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: eeeeoo
step: 6 cost: 1.5042343 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: eeeeoo
step: 7 cost: 1.4843415 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: eeeeoo
step: 8 cost: 1.4646546 prediction: [[2 2 2 2 4 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: eeeeoo
step: 9 cost: 1.4490776 prediction: [[2 2 2 2 4 4]] true Y: [[1,

In [68]:
test = [[3,1,0,2,1]]
np.squeeze(test)
test2 = [idx2char [c] for c in np.squeeze(test)]
test2


['l', 'i', 'h', 'e', 'i']

In [30]:
import numpy as np
from tensorflow.contrib import rnn
import pprint
tf.reset_default_graph()
pp=pprint.PrettyPrinter(indent=4)
sess=tf.InteractiveSession()

C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# 

In [33]:
h = [ 1,0,0,0 ]
e = [ 0,1,0,0 ]
l = [ 0,0,1,0 ]
o = [ 0,0,0,1 ]


In [51]:
with tf.variable_scope('first_cell') as scope:
    hidden_size = 2 # hidden layer에 뉴런의 개수 => 추출하고자하는 특징 
    cell=tf.contrib.rnn.BasicRNNCell(num_units = hidden_size)
    print(cell.output_size, cell.state_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,o],
                   [l,l,e,e,l]], dtype=np.float32)
    print(xdata.shape)
    # (1,5,4)
    # (batch_size, seq_len, input_dim)
    pp.pprint(xdata)
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
#     pp.pprint(_states.eval())


2 2
(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)


ValueError: Variable first_cell/rnn/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 2005, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3616, in create_op
    op_def=op_def)
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 788, in _apply_op_helper
    op_def=op_def)
  File "C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\gen_state_ops.py", line 2023, in variable_v2
    shared_name=shared_name, name=name)


In [52]:
# M(시트), V(그래프), C(컨트롤러(막대)) 분리

with tf.variable_scope('lstm_cell') as scope:
    hidden_size = 2 # hidden layer에 뉴런의 개수 => 추출하고자하는 특징 
    cell=tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)
#     print(cell.output_size, cell.state_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,o],
                   [l,l,e,e,l]], dtype=np.float32)
#     print(xdata.shape)
    # (1,5,4)
    # (batch_size, seq_len, input_dim)
    pp.pprint(xdata)
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
#     pp.pprint(_states.eval())

W0816 14:49:34.240628  9048 deprecation.py:323] From <ipython-input-52-867b6c834814>:5: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.09323087, -0.04512409],
        [ 0.25514698, -0.07846113],
        [ 0.20072179,  0.00124812],
        [ 0.20764273,  0.06136194],
        [ 0.2063224 ,  0.11119951]],

       [[ 0.19903336, -0.03877449],
        [ 0.17332144,  0.02782729],
        [ 0.20608415,  0.07954213],
        [ 0.2198289 ,  0.11319541],
        [ 0.21645439,  0.1516391 ]],

       [[ 0.08690999,  0.06686973],
        [ 0.14548315,  0.10829188],
        [ 0.32213342,  0.03794055],
        [ 0.39720687, -0.03461912],
        [ 0.28082976,  0.02777009]]], dtype=float32)


In [53]:
with tf.variable_scope('lstm_cell_2') as scope:
    hidden_size = 2 
    cell=tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,o],
                   [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(xdata)
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata, sequence_length=[5,3,4],
                                         dtype=tf.float32)
    sess.run(tf.global_variables_initializer())  
    pp.pprint(outputs.eval())

W0816 14:52:21.974403  9048 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\rnn.py:244: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.10178253,  0.09935846],
        [ 0.03285294,  0.09438521],
        [ 0.03082623,  0.0990544 ],
        [ 0.03552267,  0.0956865 ],
        [-0.08366915,  0.14030233]],

       [[ 0.11956055,  0.02453938],
        [-0.02269701,  0.05609597],
        [-0.00220019,  0.05826213],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.01948643,  0.02563055],
        [ 0.03365279,  0.04297619],
        [ 0.14485212,  0.04996748],
        [ 0.21620093,  0.04910621],
        [ 0.        ,  0.        ]]], dtype=float32)


In [54]:
with tf.variable_scope('init_lstm_cell') as scope:
    hidden_size = 2 
    batch_size = 3
    cell=tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,o],
                   [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(xdata)
    init_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    
    # 데이터 100개 / 1에폭 : 100개 데이터를 학습 
    # batch_size : 20개, batch_length : 100/20 = 5번 
    # (20개 단위로 학습) forward..reduce.mean..backward...weight, bias updating(20)
    

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.16175823,  0.10527496],
        [-0.02057432,  0.2238766 ],
        [ 0.09050904,  0.15920585],
        [ 0.1458    ,  0.07107248],
        [ 0.20420833, -0.00629528]],

       [[ 0.01012609,  0.1273628 ],
        [ 0.14429356,  0.0654773 ],
        [ 0.16485204, -0.00569641],
        [ 0.17575473, -0.07370748],
        [ 0.21216069, -0.11822865]],

       [[ 0.08908682, -0.04157171],
        [ 0.13136767, -0.08954674],
        [ 0.04509879,  0.04673268],
        [ 0.04092742,  0.15313418],
        [ 0.13335706,  0.08448996]]], dtype=float32)


In [55]:
batch_size = 3
seq_len=5
in_dim=3
xdata = np.arange(45).reshape(batch_size, seq_len, in_dim) # ohe되어져있는것
print(xdata)

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]
  [ 9 10 11]
  [12 13 14]]

 [[15 16 17]
  [18 19 20]
  [21 22 23]
  [24 25 26]
  [27 28 29]]

 [[30 31 32]
  [33 34 35]
  [36 37 38]
  [39 40 41]
  [42 43 44]]]


In [59]:
with tf.variable_scope('multi_rnn_cell') as scope:
    
    cell=tf.contrib.rnn.BasicLSTMCell(num_units = 5)
    cell=tf.contrib.rnn.MultiRNNCell([cell]*3) # layer 3개
    
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata)
    sess.run(tf.global_variables_initializer(dtype=np.float32))
    pp.print(outputs.eval())
    
#     xdata=np.array([[h,e,l,l,o],
#                    [e,o,l,l,o],
#                    [l,l,e,e,l]], dtype=np.float32)
#     pp.pprint(xdata)
#     init_state = cell.zero_state(batch_size, tf.float32)
#     outputs, _states = tf.nn.dynamic_rnn(cell, xdata, initial_s,dtype=tf.float32)
#     sess.run(tf.global_variables_initializer())
#     pp.pprint(outputs.eval())
    
    

ValueError: If there is no initial_state, you must give a dtype.

In [63]:
ydata=tf.constant([[1,1,1]])
prediction1 = tf.constant([ [[0.2, 0.7], [0.1, 0.9], [0.3, 0.7] ]])
prediction2 = tf.constant([ [[0.4, 0.6], [0.4, 0.6], [0.6, 0.4] ]])
loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, ydata)
sess.run(tf.global_variables_initializer())
loss1.eval()

# 각각의 cost 에 대하여 reduce.mean을 구한다 
# RNN 내 cost 구하기  
hihell  

TypeError: sequence_loss() missing 1 required positional argument: 'weights'